In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
!python main.py

In [ ]:
import main

In [ ]:
main.encoding()

In [ ]:
main.train_LDM()

# **Checking the data input**

In [10]:
import nibabel as nib
import numpy as np
import os

base_path = "/workspace/projects/T1-SPECT-translation/IL-CLDM/data/whole_MRI"

files = os.listdir(base_path)

for file in files:
    if file.endswith(".nii"):
        filepath = os.path.join(base_path, file)
        img = nib.load(filepath)
        data = img.get_fdata()
        print("dimansions of the file:", data.shape)
        if np.isnan(data).any():
            #print(f"El archivo {file} contiene NaNs.")
            print(".")
        else:
            print(f"El archivo {file} no contiene NaNs.")

dimansions of the file: (182, 218, 182)
El archivo 3353.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3563.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3321.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3380.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3863.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 4035.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3370.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 50028.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3120.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3310.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3389.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3188.nii no contiene NaNs.
dimansions of the file: (182, 218, 182)
El archivo 3300.nii no 

In [ ]:
import nibabel as nib, numpy as np
from pathlib import Path

def resolve(root, bid):
    for ext in ('.nii', '.nii.gz'):
        p = Path(root)/f"{bid}{ext}"
        if p.exists(): return p
    raise FileNotFoundError(bid)

def stats(path):
    data = nib.load(str(path)).get_fdata()
    data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
    return data.min(), data.max(), data.mean(), data.std()

root_abeta = Path('data/whole_SPECT')
root_mri = Path('data/whole_MRI')
train_ids = Path('data_info/train.txt').read_text().splitlines()[:-10]  # cambia [:3] por más
print('IDs', train_ids)
for bid in train_ids:
    pA = resolve(root_abeta, bid)
    pM = resolve(root_mri, bid)
    amin, amax, amean, astd = stats(pA)
    mmin, mmax, mmean, mstd = stats(pM)
    print(f"{bid} Abeta: min {amin:.4f} max {amax:.4f} mean {amean:.4f} std {astd:.4f}")
    print(f"{bid} MRI  : min {mmin:.4f} max {mmax:.4f} mean {mmean:.4f} std {mstd:.4f}")

# **Checking the PSNR AND SSIM metric**

In [11]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from dataset import OneDataset
import config

ds = OneDataset(root_Abeta=config.whole_Abeta, task=config.validation, stage="validation")
for i in range(3):  # primeros 3
    vol, name = ds[i]
    vol = vol.astype(np.float32)
    rng = max(vol.max() - vol.min(), 1e-8)
    print(name, "range", vol.min(), vol.max(),
          "PSNR self", psnr(vol, vol, data_range=rng),
          "SSIM self", ssim(vol, vol, data_range=rng))
    # Volumen con ruido para ver caída
    noise = np.random.normal(0, 0.01, size=vol.shape).astype(np.float32)
    vol_noisy = np.clip(vol + noise, 0, 1)
    print("  noisy PSNR", psnr(vol, vol_noisy, data_range=rng),
          "noisy SSIM", ssim(vol, vol_noisy, data_range=rng))

min and max of Abeta: 0.0 1.0


/opt/conda/lib/python3.10/site-packages/skimage/metrics/simple_metrics.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range**2) / err)


3358.nii range 0.0 1.0 PSNR self inf SSIM self 1.0
  noisy PSNR 40.02395494600445 noisy SSIM 0.9243681541842526
min and max of Abeta: 0.0 1.0
3320.nii range 0.0 1.0 PSNR self inf SSIM self 1.0
  noisy PSNR 40.1520364899115 noisy SSIM 0.9237057643035814
min and max of Abeta: 0.0 1.0
3160.nii range 0.0 1.0 PSNR self inf SSIM self 1.0
  noisy PSNR 40.078855880447854 noisy SSIM 0.9272966434976976


In [ ]:
import os, nibabel as nib, numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import config

ids = open(config.validation).read().split()
for bid in ids[:10]:  # revisa los primeros 10
    path = None
    for ext in (".nii", ".nii.gz"):
        p = os.path.join(config.whole_Abeta, bid + ext)
        if os.path.exists(p):
            path = p
            break
    if not path:
        print("no file for", bid)
        continue
    img = nib.load(path).get_fdata().astype(np.float32)
    vol = img[10:170, 18:210, 10:170]  # mismo crop
    vmin, vmax = vol.min(), vol.max()
    vol = np.zeros_like(vol) if vmax - vmin < 1e-8 else (vol - vmin) / (vmax - vmin)
    rng = max(vol.max() - vol.min(), 1e-8)
    psnr_self = psnr(vol, vol, data_range=rng)
    ssim_self = ssim(vol, vol, data_range=rng)
    print(bid, "range", vol.min(), vol.max(), "psnr_self", psnr_self, "ssim_self", ssim_self)
    if np.isfinite(ssim_self) and ssim_self > 0.9:
        break


# **Plotting curves of the results**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
gen_path = "/workspace/projects/T1-SPECT-translation/IL-CLDM/result/exp_1"
file1 = "loss_curve.csv"
file2 = "validation.csv"

loss_df = pd.read_csv(os.path.join(gen_path, file1))
similarity_df = pd.read_csv(os.path.join(gen_path, file2))

In [ ]:
loss_df.head()

In [ ]:
similarity_df.head()

In [ ]:
#plotting loss curve
plt.figure(figsize=(10,5))
plt.plot(loss_df['Epoch'], loss_df['recon_loss'], label='Reconstruction Loss')
plt.plot(loss_df['Epoch'], loss_df['disc_loss_epoch'], label='Discriminator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss Curve')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#plotting the similarity metrics
plt.figure(figsize=(10,5))
plt.plot(similarity_df['Epoch'], similarity_df['SSIM'], label='SSIM')
plt.plot(similarity_df['Epoch'], similarity_df['PSNR'], label='PSNR')
plt.xlabel('Epoch')
plt.ylabel('Similarity Metrics')
plt.title('Validation Similarity Metrics')
plt.legend()
plt.grid()
plt.show()

# **Converting the nii.gz into nii files**

In [ ]:
import nibabel as nib
import gzip
import shutil
from pathlib import Path

In [ ]:
input_folder = "/workspace/projects/T1-SPECT-translation/IL-CLDM/data/whole_MRI"
output_folder = "/workspace/projects/T1-SPECT-translation/IL-CLDM/data/whole_MRI2"
Path(output_folder).mkdir(parents=True, exist_ok=True)

for f in Path(input_folder).glob("*.nii.gz"):
    img = nib.load(str(f))
    out_path = Path(output_folder) / f.with_suffix('').name  # quita .gz → .nii
    nib.save(img, str(out_path))
    print("Guardado:", out_path)
